In [121]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from spkit import pfa

In [115]:
%load_ext autoreload

In [116]:
%autoreload 1

In [117]:
%aimport spkit.pfa

In [171]:
df = pd.read_csv("~/Documentos/ktm/data/dummy/data.csv")
df

,user,item,skill,correct,wins,fails
0,1,1,1,1,0,0
1,1,2,2,0,0,0
2,2,1,1,0,0,0
3,2,1,1,1,0,1
4,2,2,2,0,0,0


Onehot encoding wins and fails per skill

In [172]:
skills = df['skill'].unique()
skills_array = skills.reshape(-1, 1)
type(skills)

numpy.ndarray

In [193]:
enc = OneHotEncoder(sparse=False, categories='auto')
skills_onehot = enc.fit_transform(skills_array)
skills_onehot

array([[1., 0.],
       [0., 1.]])

In [146]:
skills_onehot[0]

array([1., 0.])

In [154]:
def create_onehot(row):
    idx = np.where(skills == row['skill'])[0]
    wins = row['wins']*skills_onehot[idx][0]
    print(wins)
    fails = row['fails']*skills_onehot[idx][0]
    return np.concatenate((skills_onehot[idx][0], wins, fails))

onehot_array = df.apply(create_onehot, axis=1)
cols = ["skills_%d"%skill for skill in skills]
cols += ["wins_%d"%skill for skill in skills]
cols += ["fails_%d"%skill for skill in skills]

[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]


In [148]:
df = pd.DataFrame(pfa_data, columns=["skill", "wins", "fails", "outcome"])

In [149]:
df

,skill,wins,fails,outcome
0,0,0,0,1
1,1,0,0,0
2,0,0,0,0
3,0,0,1,1
4,1,0,0,0


In [155]:
onehot_array

0    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
2    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
3    [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
4    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
dtype: object

In [151]:
onehot_df = pd.DataFrame(onehot_array.tolist(), columns=cols)
df = pd.concat((df,onehot_df), axis=1)
df

ValueError: Must pass 2-d input

In [9]:
X = df[cols]
X = sm.add_constant(X, prepend=False)
y = df['correct']

In [10]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

In [107]:
model = LogisticRegression(fit_intercept=False, penalty='elasticnet', solver='saga', l1_ratio=0.01)
mdl = model.fit(X, y)
model.coef_

array([[ 0.33611468, -0.59631285,  0.        ,  0.        ,  0.37940596,
         0.        , -0.25012246]])

In [108]:
model.predict_proba(X)

array([[0.47851518, 0.52148482],
       [0.69981883, 0.30018117],
       [0.47851518, 0.52148482],
       [0.38570601, 0.61429399],
       [0.69981883, 0.30018117]])

In [109]:
model.predict(X)

array([1, 0, 1, 1, 0])

In [110]:
model.coef_[0]/res.params.values

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.67294775, 0.10862896,        nan,        nan, 0.07167657,
              nan, 0.51080627])

In [111]:
model = sm.GLM(y, X, family=sm.families.Binomial())
res = model.fit_regularized(alpha=0.001)
res.params

skills_1    0.499466
skills_2   -5.489446
wins_1      0.000000
wins_2      0.000000
fails_1     5.293305
fails_2     0.000000
const      -0.489662
dtype: float64

In [112]:
predictions = res.predict(X)
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
predictions_nominal

[1, 0, 1, 1, 0]

In [87]:
predictions

0    0.502451
1    0.002525
2    0.502451
3    0.995049
4    0.002525
dtype: float64

In [25]:
res.summary()

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                correct   No. Observations:                    5
Model:                            GLM   Df Residuals:                        2
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1.3863
Date:                Tue, 15 Oct 2019   Deviance:                       2.7726
Time:                        17:07:29   Pearson chi2:                     2.00
No. Iterations:                    20                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
skills_1   -3.429e-24      1.414  -2.42e-24      1.000      -2.772       2.772
skills_2     -21.5661   2.07e+04     -0.001      0.999   -4.05e+04    4.05e+04
wins_1              0          0        nan        nan           0           0
wins_2              0          0        nan        nan           0           0
fails_1       21.5661   2.92e+04      0.001      0.999   -5.73e+04    5.73e+04
fails_2             0          0        nan        nan           0           0
==============================================================================
"""

In [178]:
q_matrix = np.asarray([[1,0], [0,1]])
data = [[1,1,0],[0,1,1],[0,2,0],[1, 2,0],[0,2,1]]

In [181]:
data

[[1, 1, 0], [0, 1, 1], [0, 2, 0], [1, 2, 0], [0, 2, 1]]

In [255]:
model = pfa.PFA()

In [263]:
internal_model = model.fit(data, q_matrix, lib="sm",penalty="elasticnet", solver="saga", l1_ratio=0.01, alpha=0.01)
internal_model.summary()

/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


NotImplementedError: 